## Importando as Bibliotecas

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import time
import os
import pandas as pd

### Monitor de download: Função que espera o download de um arquivo acabar, com um timeout específico

In [ ]:
import time

def download_wait(directory, timeout, nfiles=None):
    """
    Wait for downloads to finish with a specified timeout.

    Args
    ----
    directory : str
        The path to the folder where the files will be downloaded.
    timeout : int
        How many seconds to wait until timing out.
    nfiles : int, defaults to None
        If provided, also wait for the expected number of files.

    """
    seconds = 0
    dl_wait = True
    while dl_wait and seconds < timeout:
        time.sleep(1)
        dl_wait = False
        files = os.listdir(directory)
        if nfiles and len(files) != nfiles:
            dl_wait = True

        for fname in files:
            if fname.endswith('.crdownload'):
                dl_wait = True

        seconds += 1
    return seconds

## Inicializar classe RemuneracoesCampinas com Selenium e Chromium
#### A classe contém a conexão com o navegador e configurações, o link para o site e determinadas ações para realizar dentro do site

In [ ]:
class RemuneracoesCampinas:
    def __init__(self):
        self.site_link = "https://remuneracoes.campinas.sp.gov.br/remuneracoes/relatorio/PMCTransparenciaSalarioServidor"
        self.site_map = {
            "buttons":{
                "ano":{
                    "xpath" : "/html/body/div[3]/div[1]/form/div[4]/div/div/div[1]",
                    "xpath_selection" : "/html/body/div[3]/div[1]/form/div[4]/div/div/div[2]/div/div"
                },
                "mes":{
                    "xpath" : "/html/body/div[3]/div[1]/form/div[5]/div/div/div[1]",
                    "xpath_selection" : "/html/body/div[3]/div[1]/form/div[5]/div/div/div[2]/div/div"
                },
                "download":{
                    "xpath" : "/html/body/div[3]/div[1]/form/div[6]/div[2]/button[2]"
                }
            }
        }
        self.options = Options()
        self.options.binary_location = f"{os.getcwd()}\\chrome-win\\chrome.exe"
        self.options.add_experimental_option('prefs', {
            "download.default_directory": f"{os.getcwd()}\\Downloads_base_salarios",
        })
        self.driver = webdriver.Chrome(executable_path=f"{os.getcwd()}\\chromedriver.exe", options = self.options)

    def abrir_site(self):
        time.sleep(2)
        self.driver.get(self.site_link)
        time.sleep(7)
        
    def seleciona(self, campo, valor):
        xpath = self.site_map["buttons"][campo]["xpath"]
        xpath_selection = self.site_map["buttons"][campo]["xpath_selection"]
        self.driver.find_element("xpath", xpath).click()
        time.sleep(1)
        self.driver.find_element("xpath", xpath_selection+'[@data-value="' + valor + '"]').click()
        
    def download(self):
        self.driver.find_element("xpath", self.site_map["buttons"]["download"]["xpath"]).click()
        
    def get_driver(self): return self.driver
    def get_site_map(self): return self.site_map

In [ ]:
rc = RemuneracoesCampinas()

## Navegação pelo site

##### Variaveis: são utilizados vetores com os anos e meses possíveis para realizar o download do site

In [ ]:
anos = ["2016", "2017", "2018", "2019", "2020", "2021", "2022", "2023"]
meses = ["Janeiro", "Fevereiro", "Março", "Abril", "Maio", "Junho", "Julho", "Agosto", "Setembro", "Outubro", "Novembro", "Dezembro"]
meses_n = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]

### Manipulando site com tratamento de excessão
Dentro desse loop for, o código vai baixar, para cada ano em cada mes a base de remunerações correspondente. E renomeandoo arquivo baixado com o sufixo do ano e mes que baixou.

In [ ]:
rc.abrir_site()

In [ ]:
for ano in anos:
    rc.seleciona("ano", ano)
    for idx, mes in enumerate(meses):
        rc.seleciona("mes", mes)
        i=0
        while True:
            try:
                rc.download()
                download_wait(f"{os.getcwd()}\\Downloads_base_salarios", 180)
                os.rename(f"{os.getcwd()}\\Downloads_base_salarios\\PMCTransparenciaSalarioServidor.csv", f"{os.getcwd()}\\Downloads_base_salarios\\PMCTransparenciaSalarioServidor_"+ano+meses_n[idx]+".csv")
                break
            except:
                if(i==10):
                    time.sleep(1)
                    rc.abrir_site()
                    time.sleep(1)
                    rc.seleciona("ano", ano)
                    time.sleep(1)
                    rc.seleciona("mes", mes)
                    time.sleep(1)
                    i=0
                i+=1

### Empilhar

In [4]:
nome_diretorio = f'{os.getcwd()}\\Downloads_base_salarios'
csvs_historico = os.listdir(nome_diretorio)
for idx, arq in enumerate(csvs_historico):
    caminho_arq = nome_diretorio+"\\"+arq
    anomes = arq[32:38]
    
    if(idx==0):
        df = pd.read_csv(caminho_arq,sep=';')
        df["Anomes"] = anomes
    else:
        df_mes = pd.read_csv(caminho_arq,sep=';')
        df_mes["Anomes"] = anomes
        df = pd.concat([df,df_mes])
        
df.to_csv('base_salarios_completa.csv', index=False)